In [ ]:
import pandas as pd 
import nltk
import numpy as np
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import linear_kernel
from queue import PriorityQueue as pq

In [ ]:
# 1: Title
# 2: Release Year 
# 3: Genre
# 4: Duration
# 6: Critics
# 7: Public 
# 8: Description
# 9: Notes 
# 15: Rating ; Pg13
# 16: Listed In
# 17: Description

In [ ]:
text = open("review_ratings.csv")
output = open("res.txt","w")
# print(text)
text.readline() # remove header
for row in text: 
    row_details = row.split('^')
    title, year, genre, duration = row_details[1].strip(' ,"\''), row_details[2].strip(' ,"\''), row_details[3].strip(' ,"\''), row_details[4].strip(' ,"\'')
    critic_rate, pub_rate = row_details[6].strip(' ,"\''), row_details[7].strip(' ,"\'')
    desc, notes, listed_in, comment = row_details[8].strip(' ,"\''),row_details[9].strip(' ,"\''),row_details[16].strip(' ,"\''),row_details[17].strip(' ,"\'')
    alpha_rate = row_details[16].strip(' ,"\'')

    
    combined_desc = desc+' '+ notes+' '+listed_in+' ' +comment+' ' +title+' '+genre
    formatted_str = title+'\t'+str(year)+('\t')+str(critic_rate)+'\t'+str(pub_rate)+'\t'+combined_desc+'\n'
    output.write(formatted_str)
    # print(desc)

output.close()
text.close()


In [ ]:
punc = string.punctuation
films = {}
text = open("review_ratings.csv")
# print(text)
text.readline() # remove header
for row in text: 
    row_details = row.split('^')
    title, year, genre, duration = row_details[1].strip(' ,"\''), row_details[2].strip(' ,"\''), row_details[3].strip(' ,"\''), row_details[4].strip(' ,"\'')
    critic_rate, pub_rate = row_details[6].strip(' ,"\''), row_details[7].strip(' ,"\'')
    desc, notes, listed_in, comment = row_details[8].strip(' ,"\''),row_details[9].strip(' ,"\''),row_details[16].strip(' ,"\''),row_details[17].strip(' ,"\'')
    alpha_rate = row_details[16].strip(' ,"\'')

    combined_desc = desc+' '+ notes+' '+listed_in+' ' +comment+' ' +title+' '+genre

    valid_words = ""
    for word in combined_desc: 
        if word == " ": 
            valid_words += word
        elif word in punc: 
            pass 
        elif word.isalpha() == False: 
            pass
        else: 
            valid_words += word 
    films[(title, year)] = valid_words
            
    # formatted_str = title+'\t'+str(year)+('\t')+str(critic_rate)+'\t'+str(pub_rate)+'\t'+combined_desc+'\n'

text.close()
# films

In [ ]:
print("There are",len(films),"films in this dataset")
films_df = pd.DataFrame.from_dict(films,orient="index")
films_df.columns = ['desc']
films_df

There are 846 films in this dataset


,desc
"(Alice Doesn't Live Here Anymore, 1974)",A widow after an unfortunate marriage Alice to...
"(White Christmas, 1954)",Two former comrades who have set up a variety ...
"(Carrie, 1976)",Carrie Withe is a girl who received an obsessi...
"(The Cincinnati Kid, 1965)",Cincinnati Kid is a young champion who is maki...
"(Dad, 1989)",Jake Tremont Lemmon lives with his wife away f...
...,...
"(Minimalism: A Documentary About the Important Things, 2016)",How could our lives improve if we reduced the ...
"(Kis Kisko Pyaar Karoon, 2015)",With his three different identities Shiv Ram a...
"(Deviant Love, 2019)",Jamies married life falls apart when she finds...
"(Time Trap, 2018)",An archeology professor disappears after enter...


In [ ]:
# Remove line breaks 
def remove_linebreaks(input):
    text = re.compile(r'\n')
    return text.sub(r' ',input)

films_df["desc"] = films_df["desc"].apply(remove_linebreaks)

In [ ]:
# Tokenize words 
nltk.download('punkt')
films_df["desc"] = films_df["desc"].apply(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Remove stopwords
nltk.download('stopwords')
def remove_stopwords(input1):
    words = []
    for word in input1:
        if word not in stopwords.words('english'):
            words.append(word)
    return words
films_df["desc"] = films_df["desc"].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')
lem = WordNetLemmatizer()
def lemma_wordnet(input):
    return [lem.lemmatize(w) for w in input]
films_df["desc"] = films_df["desc"].apply(lemma_wordnet)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
def combine_text(input):
    combined = ' '.join(input)
    return combined
films_df["desc"] = films_df["desc"].apply(combine_text)

In [ ]:
# Train a TF-IDF vectorizer 
vectorizer = TfidfVectorizer(max_features=1800, lowercase=True, stop_words='english', ngram_range=(1,2)) #unigrams and bigrams

# features 
tf_idf_output = vectorizer.fit_transform(films_df["desc"])
vocab = np.array(vectorizer.get_feature_names())


/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
input_keywords = 'boy romance girl crush love'

In [ ]:
# # in input_keywords, separated by spaces, type in a few keywords about the movie you'd like to watch
# # and from our database, we'd recommend you one! 
# input_keyword = "boy romance girl crush love"
# input_keywords = input_keyword.split() + words_string.split()
# tf_idf_input_keywords = vectorizer.fit_transform(input_keywords)
# # print(tf_idf_input_keywords)
# cosine_similarities = linear_kernel(tf_idf_output, tf_idf_input_keywords)
# print(m(cosine_similarities))

# NMF

In [ ]:
num_topics = '50'

In [ ]:
num_topics = int(num_topics)
nmf = NMF(n_components = num_topics, solver = "mu").fit(tf_idf_output)
W = nmf.fit_transform(tf_idf_output)
H = nmf.components_

In [ ]:
num_top_words = '20'

In [ ]:
num_top_words = int(num_top_words)
def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [ ]:
topics = show_topics(H)
i = 1

topics_set_list = []
for row in topics: 
    topic = row.split()
    print("Topic",i,":",set(topic))
    topics_set_list.append(set(topic))
    i += 1


Topic 1 : {'trip', 'hope', 'troubled', 'international', 'relationship', 'romantic', 'run', 'movies', 'middleaged', 'dramas', 'path', 'day', 'comedies', 'independent', 'past', 'drama', 'face'}
Topic 2 : {'adventure', 'action', 'dramas', 'revenge', 'village', 'team', 'force', 'international', 'stop', 'epic', 'mission', 'explosive', 'arm', 'hunter', 'commander', 'scifi', 'battle', 'comedies', 'thor'}
Topic 3 : {'thrillers', 'evil', 'house', 'remote', 'monster', 'international', 'group', 'forest', 'movies', 'independent', 'cult', 'anthology', 'terrible', 'comedies', 'begin', 'horror', 'patient', 'haunt'}
Topic 4 : {'comedy', 'birthday', 'little', 'dj', 'friend', 'dramas', 'dating', 'feeling', 'best', 'success', 'harper', 'invite', 'crazy', 'romantic', 'business', 'trying', 'spend', 'comedies', 'end'}
Topic 5 : {'dog', 'build', 'little', 'like', 'family', 'new', 'movies', 'children', 'animation', 'child', 'world', 'comedies', 'save', 'parent', 'dramas', 'boy', 'yearold'}
Topic 6 : {'story',

In [ ]:
def jaccard_distance(topic_set, desc_set):
    intersect = len(topic_set.intersection(desc_set))
    union = len(topic_set.union(desc_set))
    return 1-intersect/union
    # return len(topic_set.intersection(desc_set))

In [ ]:
input_keyword = "family children dog little build together"
input_keywords_to_set = set(input_keyword.split())
best_topic_score = 1
best_topics = None
for topic in topics_set_list: 
    # print(topic, jaccard_distance(topic, input_keywords_to_set))
    if jaccard_distance(topic, input_keywords_to_set) < best_topic_score: 
        # print(jaccard_distance(topic, input_keywords_to_set),best_topic_score)
        best_topic_score = jaccard_distance(topic, input_keywords_to_set) 
        best_topics = topic

print(best_topics)

{'dog', 'build', 'little', 'like', 'family', 'new', 'movies', 'children', 'animation', 'child', 'world', 'comedies', 'save', 'parent', 'dramas', 'boy', 'yearold'}


In [ ]:
pq_best_titles = pq()
# max_d = 0
for k, v in films.items(): 
    jd = jaccard_distance(best_topics, set(v.split()))
    pq_best_titles.put((jd, k))

num_recs = 0
while pq_best_titles.empty() == False and num_recs < 5:
    print("Title:",pq_best_titles.get()[1][0],"\tRelease Year:",pq_best_titles.get()[1][1])
    num_recs += 1 

    

Title: Extinction 	Release Year: 2019
Title: Little Monsters 	Release Year: 2017
Title: Before I Wake 	Release Year: 2005
Title: Tarzan 	Release Year: 2013
Title: Evan Almighty 	Release Year: 2017


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=21bf25da-b52d-4747-8ce6-0a60c023609b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>